## Basic LLM

First run Ollama, then:

In [22]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama3")

After initializing the model we can start prompting it:

In [14]:
llm.invoke('how can langsmith help with testing?')

"LanguageSmith, a language translation AI model, can assist with testing in several ways:\n\n1. **Automated Translation Testing**: LanguageSmith can translate your test scenarios, user stories, or acceptance criteria into the target languages, allowing you to test your application's internationalization and localization (i18n/l10n) features.\n2. **Test Data Generation**: The AI model can generate test data in various languages, helping you to create diverse test cases for your application's language-related features.\n3. **Language-specific Testing**: LanguageSmith can assist with testing specific language features, such as:\n\t* Character encoding and decoding\n\t* Unicode support\n\t* Right-to-left (RTL) or left-to-right (LTR) text handling\n4. **Collaborative Review**: The AI model can help review and validate translations, ensuring that the translated content is accurate and consistent with the original content.\n5. **Test Case Generation**: LanguageSmith can generate test cases ba

We can also use prompt templates to guide the LLM:

In [6]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer."),
    ("user", "{input}")
])

We can now create a chain and invoke the LLM with the new instruction:

In [7]:
chain = prompt | llm

In [15]:
chain.invoke({"input": "how can langsmith help with testing?"})

"Excellent question!\n\nAs a world-class technical documentation writer, I'm excited to share some ideas on how Langsmith can assist with testing.\n\n1. **Automated Documentation Generation**: Langsmith's AI capabilities can generate high-quality documentation automatically, which saves time and ensures consistency. By integrating Langsmith with your testing framework, you can generate test reports, documentation, or even entire user manuals.\n2. **Test Case Creation**: Langsmith's natural language processing (NLP) can help create test cases based on the requirements or design documents. This reduces the effort required to develop test suites and ensures that all aspects of functionality are covered.\n3. **Automated Test Script Generation**: Langsmith can generate automated test scripts in various programming languages, such as Python or Java. This accelerates the testing process and helps ensure that tests are comprehensive and consistent.\n4. **Test Data Generation**: Langsmith's AI 

Adding an output parser to convert the received message to a simple string:

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

We can also include it into a chain:

In [10]:
chain = prompt | llm | output_parser

In [16]:
chain.invoke({"input": "how can langsmith help with testing?"})

"What a great question!\n\nAs a technical documentation writer, I've seen firsthand the importance of effective language in communicating complex technical information. Langsmith is an amazing tool that can help with testing by providing insights into how your written content is perceived by readers.\n\nHere are some ways Langsmith can assist with testing:\n\n1. **Readability scoring**: Langsmith's readability metrics give you a snapshot of how easily your content can be understood by different audiences, based on factors like sentence length, word complexity, and more. This helps you identify areas that may require simplification or rephrasing.\n2. **Grammar and syntax analysis**: Langsmith's grammar and syntax checks can help you catch mistakes or ambiguities in your writing, ensuring that your content is free of errors and flows smoothly.\n3. **Clarity evaluation**: By analyzing the structure and organization of your content, Langsmith helps you identify areas where the information 

## Retrieval Chain

Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in. A retriever can be backed by anything, in this example we'll create a vector store from a website.

In [13]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [23]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

We can now use embeddings to ingest documents into vectorstore:

In [27]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3")

In [19]:
%pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 25.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [28]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

We can now create a retrieval chain.

Take question -> Look up relevant documents -> Pass them with the question to LLM -> Ask to answer it

In [29]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

We can pass the documents ourselves:

In [30]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'According to the provided context, Langsmith can help with testing by letting you "visualize test results".'

We actually want the retriever to dinamically select the most relevant documents:

In [31]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [32]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

According to the provided context, LangSmith can help with testing in several ways:

1. **Creating datasets**: Developers can create collections of inputs and reference outputs (test cases) and use these to run tests on their LLM applications.
2. **Running custom evaluations**: LangSmith allows developers to score test results using both LLM-based and heuristic-based evaluations.
3. **Comparison view for test runs**: The platform provides a user-friendly comparison view for test runs, enabling developers to track and diagnose regressions in test scores across multiple revisions of their application.
4. **Playground environment**: Langsmith offers a playground environment for rapid iteration and experimentation, allowing developers to quickly test out different prompts and models.
5. **Automations**: Automations can be used to perform actions on traces in near real-time, such as automatically scoring traces, sending them to annotation queues, or sending them to datasets.

These features

## Conversation Retrieval Chain

To change to a chat that can answer follow-up questions we need two things:
1. The retrieval method should use the whole history
2. The LLM chain should also take the history into account

In [40]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [43]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Explain the details"
})

[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.Prototyping‚ÄãPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing ‚Äî and debug where it is failing ‚Äî is incredibly important for this phase.Debugging‚ÄãWhen developing new LLM applications, we suggest having LangSmith tracing enabl

The retriever chain is now able to get documents given that the LLM can generate a new query. Now we can continue the conversation with the new documents.

In [45]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [46]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='applications are multi-turn, meaning that they involve a series of interactions between the user and the application. LangSmith provides a threads view that groups traces from a single conversation together, making it easier to track the performance of and annotate your application across multiple turns.Was this page helpful?PreviousQuick StartNextOverviewPrototypingBeta TestingProductionCommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright ¬© 2024 LangChain, Inc.', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the bre